In [41]:
import sys
sys.path.append("../src/")
sys.path.append("../")



import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import pickle

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [2]:
# first priority is to load the mpro vectors for apple

In [19]:
args = {}
args['layer'] = 8
args['clusters'] = 5
embedding_file = '../data/processed/multipro_embeddings/layer'+ str(args['layer']) + 'clusters' + str(args['clusters']) + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=args['layer'], num_clusters=args['clusters'])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
Read in 12348 vectors of size 5 X 768


In [5]:
# initialize bert, initialize buchanan model
bert = BERTBase()
mpro_model = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')

# load mcrae model
mcrae_model = torch.load('../trained_models/model.plsr.mc_rae_real.5k.100components.500max_iters')

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpl19u502f
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [102]:
mcrae_model.feature_norms.get_features('cap_(bottle)')

['is_round',
 'is_small',
 'made_of_metal',
 'made_of_plastic',
 'is_opened_by_twisting',
 'is_closed_by_twisting',
 'used_for_sealing',
 'used_on_bottles',
 'used_for_ensuring_freshness']

In [96]:
# fetch cluster centroids for 'apple'
word = 'ahead'

word_emb = embs.get_embedding(word)

In [73]:


for word_centroid in word_emb:
    # take each prototype in the bundle and predict features separately
    #predict_for_single_context_vector(apple_centroid, bert, mpro_model)
    feats = mpro_model.predict_top_n_features_from_single_context_vector(word, 10, word_centroid, output_vec=None)
    print(feats)

('ahead', ['happen', 'direct', 'time', 'before', 'future', 'after', 'look', 'end', 'present', 'move'])
('ahead', ['present', 'high', 'behind', 'move', 'future', 'front', 'time', 'direct', 'car', 'before'])
('ahead', ['front', 'part', 'direct', 'behind', 'before', 'finish', 'time', 'act', 'future', 'move'])
('ahead', ['know', 'move', 'before', 'run', 'front', 'win', 'fast', 'high', 'car', 'direct'])
('ahead', ['front', 'right', 'after', 'direct', 'behind', 'future', 'before', 'time', 'move', 'act'])


In [74]:
for word_centroid in word_emb:
    # take each prototype in the bundle and predict features separately
    #predict_for_single_context_vector(apple_centroid, bert, mpro_model)
    feats = mcrae_model.predict_top_n_features_from_single_context_vector(word, 10, word_centroid, output_vec=None)
    print(feats)

('ahead', ['is_fast', 'has_wings', 'used_for_transportation', 'made_of_plastic', 'has_a_beak', 'has_feathers', 'a_bird', 'is_long', 'different_colours', 'is_brown'])
('ahead', ['is_fast', 'has_a_beak', 'has_feathers', 'used_for_transportation', 'is_long', 'different_colours', 'a_bird', 'is_brown', 'made_of_plastic', 'made_of_wood'])
('ahead', ['an_animal', 'different_sizes', 'different_colours', 'is_edible', 'has_legs', 'is_round', 'has_4_legs', 'is_brown', 'is_thin', 'is_long'])
('ahead', ['has_wheels', 'is_red', 'is_fast', 'used_for_transportation', 'made_of_wood', 'different_colours', 'made_of_metal', 'a_bird', 'has_feathers', 'is_round'])
('ahead', ['is_edible', 'made_of_material', 'is_thin', 'is_brown', 'has_4_legs', 'is_round', 'an_animal', 'is_long', 'different_sizes', 'different_colours'])


In [75]:
# You need to look at the words that form these clusters... do they match with the description?

In [76]:
pd.option_context('display.max_rows', 5,
                       'display.max_columns', None,
                       'display.width', 1000,
                       'display.precision', 3,
                       'display.colheader_justify', 'center')

def read_clusters(word):   
    cluster_path = os.path.join('../data/interim/multipro_embeddings/', word, 'analysis_results', 'clusters.p')
    """
     this is a list of dicts with the structure:
                    {'word': tokens[0]['word'],
                    'layer': layer,
                    'k_clusters': k,
                    'cluster_id': cluster_index,
                    'centroid': cluster_centroids[cluster_index],
                    'sentence_uids': sentence_uids,
                    'within_cluster_variance': cluster_variance
                    }
    """
    data = pickle.load(open(cluster_path, 'rb'))
    # the 'if item' removes None values for cluster sizes we didnt have enough tokens for
    data = [item for sublist in data if sublist for item in sublist]
    
    columns = ['word', 'layer', 'k_clusters', 'cluster_id', 'centroid', 'sentence_uids', 'within_cluster_variance', 'average_pairwise_token_distance']
    df = pd.DataFrame.from_records(data, columns=columns)
    return df


def get_sentences_in_cluster(sentence_uids, word):
    #print(sentence_uids)
    tokens = []

    with open('../data/interim/multipro_embeddings/'+word+'/BNC_tokens.csv', mode="r") as infile:
        fieldnames = ['word', 'sentence', 'tag', 'bnc_index']
        reader = csv.DictReader(infile, delimiter="\t", fieldnames=fieldnames)
        for row in reader:
            if row['bnc_index'] in sentence_uids:
                tokens.append(row['sentence'])
    return tokens
        
# pickle_file = '../data/interim/multipro_embeddings/'+word+'/analysis_results/clusters.p'
# data = pickle.load(pickle_file)






print(word)
clusters = read_clusters(word)
#print(clusters)
clusters = clusters[clusters['k_clusters'] == 5]
clusters = clusters[clusters['layer'] == 8]
#print(clusters)


# for each of the clusters, go through the tokens we hae for that word and select the tokens that match the sentence uid

for index, cluster in clusters.iterrows():
    sentence_uids = cluster['sentence_uids']
    tokens = get_sentences_in_cluster(sentence_uids, word)
    cluster_id = cluster['cluster_id']
    data = [[word, cluster_id, token] for token in tokens]
    df = pd.DataFrame(data, columns = ['word', 'cluster', 'context'])
    
    #print(cluster['cluster_id'])
    #print(tokens)

    #for token in tokens:
    #    print(token)
    print(df)

ahead
    word   cluster  \
0   ahead     0      
1   ahead     0      
2   ahead     0      
3   ahead     0      
4   ahead     0      
5   ahead     0      
6   ahead     0      
7   ahead     0      
8   ahead     0      
9   ahead     0      
10  ahead     0      
11  ahead     0      
12  ahead     0      
13  ahead     0      
14  ahead     0      
15  ahead     0      
16  ahead     0      
17  ahead     0      
18  ahead     0      
19  ahead     0      
20  ahead     0      
21  ahead     0      
22  ahead     0      
23  ahead     0      
24  ahead     0      
25  ahead     0      
26  ahead     0      
27  ahead     0      
28  ahead     0      
29  ahead     0      
30  ahead     0      
31  ahead     0      
32  ahead     0      
33  ahead     0      
34  ahead     0      
35  ahead     0      
36  ahead     0      
37  ahead     0      
38  ahead     0      
39  ahead     0      
40  ahead     0      
41  ahead     0      
42  ahead     0      
43  ahead     0      
44  